In [8]:
using Pkg
using DifferentialEquations, Plots

    Updating registry at `C:\Users\Micha\.julia\registries\General.toml`
   Resolving package versions...
   Installed LERC_jll ───────────────────── v3.0.0+1
   Installed GR_jll ─────────────────────── v0.72.7+0
   Installed Libmount_jll ───────────────── v2.35.0+0
   Installed Opus_jll ───────────────────── v1.3.2+0
   Installed x265_jll ───────────────────── v3.5.0+0
   Installed JpegTurbo_jll ──────────────── v2.1.91+0
   Installed libfdk_aac_jll ─────────────── v2.0.2+0
   Installed LoggingExtras ──────────────── v1.0.0
   Installed RelocatableFolders ─────────── v1.0.0
   Installed Xorg_xkbcomp_jll ───────────── v1.4.6+0
   Installed Measures ───────────────────── v0.3.2
   Installed ConcurrentUtilities ────────── v2.2.0
   Installed Unitful ────────────────────── v1.15.0
   Installed ColorSchemes ───────────────── v3.21.0
   Installed Contour ────────────────────── v0.6.2
   Installed Formatting ─────────────────── v0.4.2
   Installed Grisu ──────────────────────── v1.0.2
   Ins

        Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated -m`
Precompiling project...
  ✓ Pipe
  ✓ LaTeXStrings
  ✓ TensorCore
  ✓ Contour
  ✓ SimpleBufferStream
  ✓ Measures
  ✓ Formatting
  ✓ URIs
  ✓ ConcurrentUtilities
  ✓ Unzip
  ✓ Grisu
  ✓ ExceptionUnwrapping
  ✓ BitFlags
  ✓ DelimitedFiles
  ✓ Scratch
  ✓ LoggingExtras
  ✓ TranscodingStreams
  ✓ Graphite2_jll
  ✓ OpenSSL_jll
  ✓ Libmount_jll
  ✓ LLVMOpenMP_jll
  ✓ Xorg_libXau_jll
  ✓ Bzip2_jll
  ✓ UnicodeFun
  ✓ libpng_jll
  ✓ libfdk_aac_jll
  ✓ LAME_jll
  ✓ LERC_jll
  ✓ fzf_jll
  ✓ Xorg_libXdmcp_jll
  ✓ JpegTurbo_jll
  ✓ Ogg_jll
  ✓ x265_jll
  ✓ x264_jll
  ✓ libaom_jll
  ✓ Zstd_jll
  ✓ Expat_jll
  ✓ LZO_jll
  ✓ Xorg_xtrans_jll
  ✓ Opus_jll
  ✓ Libffi_jll
  ✓ Libiconv_jll
  ✓ FixedPointNumbers
  ✓ Libgpg_error_jll
  ✓ Showoff
  ✓ RelocatableFolders
  ✓ Xorg_libpthread_stubs_jll
  ✓ Libuuid_jll
  ✓ Wayland_protocols_jll
  ✓ CodecZ

In [2]:
S = 60;
b1 = 20*(rand(S,1).- 1/2); # 3; 
b2 = 1; # b1/3; 
pc = 0.6; sig2 = 1;
K = 1;
al = 5;
fl = 1; # stabilize p
p_opt_vec = 1.4*ones(S,1) + 1*randn(S,1);
lam = 1e-5;

In [ ]:
Gamma_p(p,p_opt,pc,sig2) = 1 - (1 .- exp.(-(p-p_opt).^2 / 2 ./ sig2)) ./ ( 1-exp.(-pc^2/ 2 ./ sig2));
dn_dt_p(n,p,al,K,p_opt_vec,pc,sig2,lam) = al*n.*(1 - n/K + Gamma_p(p,p_opt_vec,pc,sig2)) + lam;
dp_dt(n,p,b1,b2,al,K,p_opt_vec,pc,sig2,fl) = al/S*sum( b1.*dn_dt_p(n,p,al,K,p_opt_vec,pc,sig2,lam).*(dn_dt_p(n,p,al,K,p_opt_vec,pc,sig2,lam)>0) ) + al/S*b2*sum(n) - fl*(p-1);

dnp_dt(t,np) =
   [(dn_dt_p(np(1:S),np(S+1),al,K,p_opt_vec,pc,sig2,lam))
   dp_dt(np(1:S),np(S+1),b1,b2,al,K,p_opt_vec,pc,sig2,fl)];

n0 = rand(S,1); n0 = n0/sum(n0) * K/2;
np0 = [n0 ; 1];

In [ ]:
[T_now,NPt_now] = ode45(dnp_dt,0:0.1:240,np0,options);
plot(T_now,NPt_now(:,1:S)), hold on
plot(T_now,NPt_now(:,end),'k','LineWidth',1), hold off